In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __init__ import *

PROJECT_JSON = kraft.read_json("../project.json")

PATH = make_path_dict(PROJECT_JSON)

In [ ]:
gps_map = kraft.read_gps_map(PATH["gps_map.pickle.gz"])

In [ ]:
feature_data_dicts = PROJECT_JSON["feature_data"]

feature_data_dicts = {
    data_name: data_dict
    for data_name, data_dict in feature_data_dicts.items()
    if "indices_to_peek" in data_dict
}

for data_name, data_dict in feature_data_dicts.items():

    print(data_name)

    data_dict["dataframe"] = pd.read_csv(data_dict["file_path"], sep="\t", index_col=0)

In [ ]:
sample_data_dicts = PROJECT_JSON["sample_data"]

sample_data_dicts = {
    data_name: data_dict
    for data_name, data_dict in sample_data_dicts.items()
    if "indices_to_peek" in data_dict
}

for data_name, data_dict in sample_data_dicts.items():

    print(data_name)

    data_dict["dataframe"] = pd.read_csv(data_dict["file_path"], sep="\t", index_col=0)

In [ ]:
for w_or_h, data_dicts in (("w", feature_data_dicts), ("h", sample_data_dicts)):

    element_summary = PROJECT_JSON["{}_summary".format(w_or_h)]

    for factor_name, data_name_indices in element_summary.items():

        for data_name, indices in data_name_indices.items():

            if data_dicts[data_name]["indices_to_peek"] != "all":

                data_dicts[data_name]["indices_to_peek"] += indices

In [ ]:
for element_type, w_or_h, elements, element_label, colorscale, data_dicts in (
    (
        "feature",
        "w",
        gps_map.w_elements,
        gps_map.w_element_label,
        gps_map.w_label_colorscale,
        feature_data_dicts,
    ),
    (
        "sample",
        "h",
        gps_map.h_elements,
        gps_map.h_element_label,
        gps_map.h_label_colorscale,
        sample_data_dicts,
    ),
):

    for data_name, data_dict in data_dicts.items():

        indices_to_peek = data_dict["indices_to_peek"]

        if indices_to_peek == "all":

            indices_to_peek = data_dict["dataframe"].index

        data_type = data_dict["data_type"]

        element_marker_size = PROJECT_JSON[
            "gps_map_{}_element_marker_size".format(w_or_h)
        ]

        if element_label is None:

            gps_map_directory_path = PATH["{}|gps_map/".format(w_or_h)]

        else:

            gps_map_directory_path = PATH["{}|hcc|gps_map/".format(w_or_h)]

        element_alias = PROJECT_JSON["{}_alias".format(element_type)]

        for index, element_value in (
            data_dict["dataframe"]
            .loc[data_dict["dataframe"].index & indices_to_peek]
            .iterrows()
        ):

            gps_map_file_name = kraft.normalize_file_name(
                "gps_map.{}.{}.html".format(data_name, index)
            )

            print(gps_map_file_name)

            gps_map.plot(
                w_or_h,
                element_value=element_value,
                element_value_data_type=data_type,
                element_marker_size=element_marker_size,
                html_file_path=os.path.join(gps_map_directory_path, gps_map_file_name),
            )

            if element_label is None:

                continue

            label_element_value = element_value.groupby(by=element_label)

            layout = {
                "title": {"text": "{}: {}".format(data_name, index)},
                "xaxis": {
                    "dtick": 1,
                    "title": {"text": "{} Cluster".format(element_alias)},
                },
                "yaxis": {"title": {"text": "Value"}},
            }

            html_file_path = os.path.join(
                PATH["{}|hcc|comparison/".format(w_or_h)], gps_map_file_name
            )

            label_color = {
                label: kraft.get_colorscale_color(
                    colorscale, label, len(label_element_value)
                )
                for label in label_element_value.groups.keys()
            }

            if data_type == "binary":

                kraft.plot_plotly_figure(
                    {
                        "layout": layout,
                        "data": [
                            {
                                "type": "bar",
                                "name": label,
                                "x": (label,),
                                "y": (element_value_.sum() / element_value_.size,),
                                "marker": {"color": label_color[label]},
                            }
                            for label, element_value_ in label_element_value
                        ],
                    },
                    html_file_path,
                )

            else:

                kraft.plot_plotly_figure(
                    {
                        "layout": layout,
                        "data": [
                            {
                                "type": "violin",
                                "name": label,
                                "y": element_value_,
                                "marker": {"color": label_color[label]},
                                "scalemode": "count",
                                "meanline": {"visible": True},
                                "points": "all",
                            }
                            for label, element_value_ in label_element_value
                        ],
                    },
                    html_file_path,
                )